# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#menu1">Download and Explore Dataset</a>

2. <a href="#menu2">Explore Neighborhoods in New York City</a>

3. <a href="#menu3">Analyze Each Neighborhood</a>

4. <a href="#menu4">Cluster Neighborhoods</a>

5. <a href="#menu5">Examine Clusters</a>    
</font>
</div>

Importing libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import geopandas as pd #geospatial data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'geopandas'

<a id='menu1'></a>

## 1. Download and Explore Dataset

<a id='menu2'></a>

## 2. Explore Neighborhoods in City of Toronto

<a id='menu3'></a>

## 3. Analyze Each Neighborhood

<a id='menu4'></a>

## 4. Cluster Neighborhoods

<a id='menu5'></a>

## 5. Examine Clusters